# WhatsApp sqlite DB tools

### Original Medium post: https://medium.com/@1522933668924/extracting-whatsapp-messages-from-backups-with-code-examples-49186de94ab4
#### License: Free to use, include credit / attribution
Please note that message information was intentionally removed from this notebook and the cells without outputs are not such by mistake

In [1]:
import pandas as pd
import sqlite3

## First, Let's figure out what tables do we have in the DB

In [21]:
con = sqlite3.connect("ChatStorage.sqlite")
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", con)
con.close()
tables

,name
0,ZWABLACKLISTITEM
1,ZWACHATPROPERTIES
2,ZWACHATPUSHCONFIG
3,ZWAMESSAGEINFO
4,Z_METADATA
5,Z_MODELCACHE
6,ZWAGROUPINFO
7,ZWAMESSAGEDATAITEM
8,ZWAPROFILEPICTUREITEM
9,ZWAGROUPMEMBERSCHANGE


## Since I already looked through those tables I know that the interesting one is ZWAMESSAGE, let's open it and start looking around

In [ ]:
con = sqlite3.connect("ChatStorage.sqlite")
df = pd.read_sql_query("SELECT * from ZWAMESSAGE", con)
con.close()
df.tail(2)

### Let's see what columns are there?

In [19]:
print('\n'.join([str(i+1)+' '+x for i, x in enumerate(df.columns)]))

1 Z_PK
2 Z_ENT
3 Z_OPT
4 ZCHILDMESSAGESDELIVEREDCOUNT
5 ZCHILDMESSAGESPLAYEDCOUNT
6 ZCHILDMESSAGESREADCOUNT
7 ZDATAITEMVERSION
8 ZDOCID
9 ZENCRETRYCOUNT
10 ZFILTEREDRECIPIENTCOUNT
11 ZFLAGS
12 ZGROUPEVENTTYPE
13 ZISFROMME
14 ZMESSAGEERRORSTATUS
15 ZMESSAGESTATUS
16 ZMESSAGETYPE
17 ZSORT
18 ZSPOTLIGHTSTATUS
19 ZSTARRED
20 ZCHATSESSION
21 ZGROUPMEMBER
22 ZLASTSESSION
23 ZMEDIAITEM
24 ZMESSAGEINFO
25 ZPARENTMESSAGE
26 ZMESSAGEDATE
27 ZSENTDATE
28 ZFROMJID
29 ZMEDIASECTIONID
30 ZPHASH
31 ZPUSHNAME
32 ZSTANZAID
33 ZTEXT
34 ZTOJID


## Now that we have a DataFrame let's start by searching based on the phone number

In [77]:
#get df that only contains chats with a specific phone number (or part of it)
get_df_by_number = lambda df, num: df[df.ZTOJID.str.contains(num).fillna(False) | df.ZFROMJID.str.contains(num).fillna(False)]
#note that the fillna is critical to prevent the weird False | None != None | False behavior

In [78]:
df2 = get_df_by_number(df, '5027')

In [79]:
#let's check how many messages did we find
len(df2)

24

In [ ]:
#let's see the text
list(df2.ZTEXT)

## Ok, it works. Let's add some tools to extract interesting messages and test them

In [81]:
#let's make a few more accessors
get_df_by_contact_name = lambda df, name: df[df.ZPUSHNAME.str.contains(name).fillna(False)]
get_df_by_chat_session_id = lambda df, sid: df[df.ZCHATSESSION == sid]
grep_for_message_text = lambda df, txt: df[df.ZTEXT.str.contains(txt).fillna(False)]

In [ ]:
get_df_by_contact_name(df, 'Emma').head(2)

In [95]:
#Let's find all chat sessions that a specific contact participated in
get_df_by_contact_name(df, 'Emma').ZCHATSESSION.unique()

array([159.])

In [ ]:
#now let's read a specific chat session
get_df_by_chat_session_id(df, 104.0).head(2)

In [ ]:
grep_for_message_text(df, 'URGENT').head(2)

## Ok, next step - let's go figure out how to format dates

In [101]:
from datetime import datetime

In [102]:
#Let's see if it just works if we try to convert ZWMESSAGEDATE to datetime assuming it is a timestamp
datetime.fromtimestamp(df4.ZMESSAGEDATE.iloc[0])

datetime.datetime(1985, 2, 6, 17, 55, 39, 331183)

### Well, that didn't work - how could there be a message from 1985? -- reading around it seems Apple likes to count time on iPhones from 1.1.2001 - Let's see what happens if we add that to our timestamp?

In [104]:
timestamp_to_apple = lambda x: datetime.fromtimestamp(x) + (datetime(2001,1,1) - datetime.fromtimestamp(0))
timestamp_to_apple(df4.ZMESSAGEDATE.iloc[0])

datetime.datetime(2016, 2, 7, 15, 55, 39, 331183)

### It works! Let's add a 'Date' column to our DB to show readable dates and make it the index

In [ ]:
df['Date'] = df.ZMESSAGEDATE.apply(timestamp_to_apple)
df.index = df.Date    # let's make a date index
df = df.sort_index()  # now let's sort by index
df.head()

### hmmm, I got messages from 2012? with no text? 
After looking around it seems to be that these messages indicate the creation dates of chat sessions or groups and are probably received from WhatsApp servers...

## Ok let's try to print everything in a nice readable format

In [ ]:
#get the dataframe of all messages where contact Emma appears and show the top 5
#values in the ZTEXT (message text) column together with index (which we set to date)
get_df_by_chat_session_id(df, 159.0)[['ZTEXT']].head() 

### it is a bit annoying that we can't see the full text, so let's fix it

In [ ]:
pd.set_option('display.max_colwidth', -1)
get_df_by_chat_session_id(df, 159.0)[['ZTEXT']].head() 

# The end - hope it was useful. If it did, please star / fork this repo :)